In [7]:
from yahoo_oauth import OAuth1

yf = OAuth1(None, None, from_file='oauth.json')
if not yf.token_is_valid():
    yf.refresh_access_token()

[2015-08-03 06:54:35,939 DEBUG] [yahoo_oauth.yahoo_oauth.__init__] Checking 
[2015-08-03 06:54:35,940 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 973.7816860675812
[2015-08-03 06:54:35,941 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID
[2015-08-03 06:54:35,943 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 973.7844531536102
[2015-08-03 06:54:35,943 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


In [98]:
print(yf.session.params)
print(yf.session.signature.NAME)
print(yf.session.signature.sign)
print(yf.session.access_token)
print(yf.oauth_version)

# useful thread
#https://developer.yahoo.com/forum/OAuth-General-Discussion-YDN-SDKs/OAuth-two-legged-documentation-omission-/1253773254000-104aac85-a511-3985-87b8-36f430730ca4/


{}
HMAC-SHA1
<bound method HmacSha1Signature.sign of <rauth.oauth.HmacSha1Signature object at 0x10e0cb978>>
A=X0yV8XSehlwTsHg7Lzv6ZdDQrERYhOcZV9j9aEqRa9cAxoi9rjkeYHxW2vViJrkdcPA6fgdODY6_Zh0UT9tMTTo_I6XHJmv2SWDmbGjg.T3PXeGIwL672Jcyp27P7Sv2tD8f8IE0dorF2o4YcUkVB1NXMLAo0mSZ9cyxrz56h9YBfExPv9rwrrtUZbMug3d1GUa2yiTuOZ3sw8t7PODWRtEK0AmLMbBBW30x1fEIlRLrWFzhlpVQ.i3eyMWkgFfjRtf0lS.tzE..lMHRZwiz8FCQGtXSnr9OCuPGDyL6il4GGDlnM2zToRF7t49D1ncJvTYN__WfCHG8cq49jy1H5uQrLAVSI_Ufw988oN5SFp3Q7QKVXQZ1HGQIk7YRbUxlPD10Kn5elKmOAFGVj0ZwEwG.cEqm7mDB7KL74d.76GvnC8FGwULugJCXsy2UdTQAzMR2xNlCahLB9P6LOG4kARpzqJexVfU.fA6i4SPYzK7t32wnx0NvCQdUZYRxeiS9xPRgBznSZ_ekECwFS6yeCoNb72gWvMqFOel7omOuUVvQpdaKeOIuv1xQ8uK8d3TtgPbdadglE8UOLXV8gVHxtVMy1KEmNNrRfBOAEoE6Nl.mXlNEzBVlm3n9fPc5Vcbi_gnB_SBUIxP3BMCY7RErzOtfU3xxrzColxjA0J0Fd6yOd7yGBgDkgOkPE4QprQgraWIBY88YqaqOxGxoZzdS1b1qgqhZEO1.cSEc2.D7lyx.JO3jr5pfBUJeDZa8BJxMKU2bwStouJcU9JN7xCFBnu_OVoXe.tpWSyPmMCraho5nIn619bUz7z2gnCW2XbZX1Kd2Fb1mivRv4D.C9F2eRAyQ8U4IZGr5DJKZ0lG2qdBMhMvHaZFrvl0ISm

# Two-step authentication process

1. Send the following url:
https://api.login.yahoo.com/oauth/v2/get_request_token?oauth_callback=oob&oauth_consumer_key=PQRST&oauth_nonce=noncasdfe&oauth_signature=ABCD%26&oauth_signature_method=PLAINTEXT&oauth_timestamp=1401800435&oauth_version=1.0
while replacing ABCD as Consumer Secret; PQRST as Consumer Key.
Note: oauth_signature is your Consumer Secret followed by %26. If you Consumer Secret was ABCD your oauth_signature would become: ABCD%26

2. You get back a string like this:
oauth_token=hxtv8zb&oauth_token_secret=aab928dd739072ba6c67abc70282e5b7e915f955&oauth_expires_in=3600&xoauth_request_auth_url=https%3A%2F%2Fapi.login.yahoo.com%2Foauth%2Fv2%2Frequest_auth%3Foauth_token%3Dhxtv8zb&oauth_callback_confirmed=true

I extracted the oauth_token_secret and the oauth_token( tho some say you dont need oauth_token). I don't need the xoauth_request_auth_url. (I think that is what I would need to pass to the user if it was for a three-legged authentication).

The oauth_signature is now : your Consumer Secret with a %26 and the value of oauth_token_secret. If your Consumer Secret was ABCD and the oauth_token_secret sent back from yahoo was ZYXABC your oauth_signature would now be: ABCD%26ZYXABC

3. Finally, you send this:
https://query.yahooapis.com/v1/public/yql?oauth_consumer_key=PQRST&oauth_nonce=UVA4JXRq4V&oauth_signature_method=PLAINTEXT&oauth_signature=ABCD%26ZYXABC&oauth_timestamp=1401868147&oauth_version=1.0&oauth_token=hxtv8zb&q=select%20*%20from%20local.search%20where%20query%3D%22sushi%22%20and%20location%3D%22san%20francisco%2C%20ca%22&format=json&diagnostics=true&callback=top_stories

In [6]:
"""
https://api.login.yahoo.com/oauth/v2/get_request_token?oauth_callback=oob&oauth_consumer_key=PQRST&oauth_nonce=noncasdfe&oauth_signature=ABCD%26&oauth_signature_method=PLAINTEXT&oauth_timestamp=1401800435&oauth_version=1.0
while replacing ABCD as Consumer Secret; PQRST as Consumer Key.
Note: oauth_signature is your Consumer Secret followed by %26. If you Consumer Secret was ABCD your oauth_signature would become: ABCD%26
"""     
# Build URL
url_base = 'https://api.login.yahoo.com/oauth/v2/get_request_token?oauth_callback=oob'
consumer_key = '&oauth_consumer_key='+ yf.session.consumer_key
consumer_secret = '&oauth_signature=' + yf.session.consumer_secret + '%26'
other = '&oauth_nonce=noncasdfe' + '&oauth_signature_method=PLAINTEXT&oauth_timestamp=1401800435&oauth_version=1.0'
print(url_base + consumer_key + consumer_secret + other)

https://api.login.yahoo.com/oauth/v2/get_request_token?oauth_callback=oob&oauth_consumer_key=dj0yJmk9VWo0RXhCSk80MmlQJmQ9WVdrOVNXRjFOa2cwTm1zbWNHbzlNQS0tJnM9Y29uc3VtZXJzZWNyZXQmeD1hYw--&oauth_nonce=noncasdfe&oauth_signature=0d0c9222ce8533a5e1350e38f9216ef633561289%26&oauth_signature_method=PLAINTEXT&oauth_timestamp=1401800435&oauth_version=1.0


In [1]:
"""
oauth_token=bkxykvs&oauth_token_secret=c99b53a5858706567a78156afb872f01c85c624d&oauth_expires_in=3600&xoauth_request_auth_url=https%3A%2F%2Fapi.login.yahoo.com%2Foauth%2Fv2%2Frequest_auth%3Foauth_token%3Dbkxykvs&oauth_callback_confirmed=true
I extracted the oauth_token_secret and the oauth_token( tho some say you dont need oauth_token). I don't need the xoauth_request_auth_url. (I think that is what I would need to pass to the user if it was for a three-legged authentication).
The oauth_signature is now : your Consumer Secret with a %26 and the value of oauth_token_secret. If your Consumer Secret was ABCD and the oauth_token_secret sent back from yahoo was ZYXABC your oauth_signature would now be: ABCD%26ZYXABC
"""
import time
import random
import base64
def get_nonce():
    """Unique token generated for each request"""
    n = base64.b64encode(
        ''.join([str(random.randint(0, 9)) for i in range(24)]).encode('utf-8'))
    return n.decode('utf-8')


oauth_token_secret = 'c99b53a5858706567a78156afb872f01c85c624d'
oauth_token = 'bkxykvs'

#http://query.yahooapis.com/v1/public/yql?q=select%20*%20from%20geo.placefinder%20where%20text%3D%2210%20prospect%20brooklyn%20ny%22%20and%20flags%3D%22GT%22%20and%20gflags%3D%22AC%22
request_base = 'http://query.yahooapis.com/v1/public/yql?'
consumer_key = 'oauth_consumer_key=' + yf.session.consumer_key
oauth_nonce = '&oauth_nonce=' + str(get_nonce())
oauth_signature_method = '&oauth_signature_method=PLAINTEXT'
oauth_signature = '&oauth_signature=' + yf.session.consumer_secret + '%26' + oauth_token_secret
oauth_timestamp = '&oauth_timestamp=' + str(int(time.time()))
oauth_version = '&oauth_version=1.0'
request_oauth_token = '&oauth_token=' + oauth_token
request_q = '&q=select%20*%20from%20geo.placefinder%20where%20text%3D%2210%20prospect%20brooklyn%20ny%22%20and%20flags%3D%22GT%22%20and%20gflags%3D%22AC%22'
print(request_base + consumer_key + oauth_nonce + 
     oauth_signature_method + oauth_signature +
     oauth_timestamp + oauth_version +
     request_oauth_token + request_q)

NameError: name 'yf' is not defined

In [127]:
print(yf.oauth.base_url)
print(yf.oauth.access_token_url)
print(yf.oauth.authorize_url)
print(yf.oauth.name)
print(yf.oauth.consumer_key)
print(yf.oauth.access_token_url)
print(yf.oauth.access_token_response.url)
print(yf.oauth.access_token_response.json)
print(yf.oauth.request_token_response.url)

None
https://api.login.yahoo.com/oauth/v2/get_token
https://api.login.yahoo.com/oauth/v2/request_auth
yahoo
dj0yJmk9VWo0RXhCSk80MmlQJmQ9WVdrOVNXRjFOa2cwTm1zbWNHbzlNQS0tJnM9Y29uc3VtZXJzZWNyZXQmeD1hYw--
https://api.login.yahoo.com/oauth/v2/get_token
https://api.login.yahoo.com/oauth/v2/get_token?oauth_signature_method=HMAC-SHA1&oauth_verifier=24h7nt&oauth_version=1.0&oauth_signature=fvgDiKAdCO%2BD%2BTroqWQZ1APhdPY%3D&oauth_token=bxvg6s2&oauth_consumer_key=dj0yJmk9VWo0RXhCSk80MmlQJmQ9WVdrOVNXRjFOa2cwTm1zbWNHbzlNQS0tJnM9Y29uc3VtZXJzZWNyZXQmeD1hYw--&oauth_timestamp=1438527474&oauth_nonce=36d18b08edb22cec0eda3b34ff0f57bd7fdf3eee
<bound method Response.json of <Response [200]>>
https://api.login.yahoo.com/oauth/v2/get_request_token?oauth_signature_method=HMAC-SHA1&oauth_version=1.0&oauth_signature=2J6JP61QnCIftsqHKeXAvaXgoBw%3D&oauth_consumer_key=dj0yJmk9VWo0RXhCSk80MmlQJmQ9WVdrOVNXRjFOa2cwTm1zbWNHbzlNQS0tJnM9Y29uc3VtZXJzZWNyZXQmeD1hYw--&oauth_callback=oob&oauth_timestamp=1438527456&oauth_non

In [135]:
r = yf.session.

None


In [68]:
#print(oauth.session.access_token)
params = {
         'consumer_key': yf.session.consumer_key,
         'consumer_secret': yf.session.consumer_secret
         }
params = {}
print(params)
url = 'https://query.yahooapis.com/v1/yql?q=select%20*%20from%20local.search%20where%20query%3D%22sushi%22%20and%20location%3D%22san%20francisco%2C%20ca%22&format=json&diagnostics=true&callback='
t1 = '%26consumer_secret%3D'+yf.session.consumer_key
t2 = '%26token_secret%3D'+yf.session.access_token_secret
responses = yf.session.get(url, 
              params=params, verify=True)

for row in responses:
    print(row)

{}
b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><yahoo:error xmlns:yahoo="http://www.yahooapis.com/v1/base.rng" yahoo:uri'
b'="http://yahoo.com" xml:lang="en-US"><yahoo:description>Please provide valid credentials. OAuth oauth_problem=&quot;OST_OAUTH_SI'
b'GNATURE_INVALID_ERROR&quot;, realm=&quot;yahooapis.com&quot;</yahoo:description><yahoo:detail>Please provide valid credentials. '
b'OAuth oauth_problem=&quot;OST_OAUTH_SIGNATURE_INVALID_ERROR&quot;, realm=&quot;yahooapis.com&quot;</yahoo:detail></yahoo:error>'


In [69]:
231   def yql(self, query):                                                         
232     parameters = { 'q': query, 'format': 'json', 'env': yql.DATATABLES_URL }    
233     request = oauthlib.oauth.OAuthRequest.from_consumer_and_token(self.consumer, token=self.token, http_method='GET', http_url=yql.OAUTH_API_URL, parameters=parameters)
234     request.sign_request(self.signature_method_hmac_sha1, self.consumer, self.token)
235     try:                                                                        
236       return simplejson.loads(self.client.access_resource(request))             
237     except:                                                                     
238       return False    

SyntaxError: invalid syntax (<ipython-input-69-e8fa04f80268>, line 1)

In [ ]:
import requests


class Request(object):
    """Class is responsible for prepare request query and sends it to YQL Yahoo API."""
    parameters = {
        'q': '',
        'format': 'json',
        'diagnostics': 'false',
        'env': 'store://datatables.org/alltableswithkeys',
        'callback': ''
    }
    api = 'https://query.yahooapis.com/v1/public/yql'https://simplypython.files.wordpress.com/2015/03/python-integrated-stock-retrieval-and-filter.jpg
    #api = 'https://query.yahooapis.com/v1/yql'
    
    def prepare_query(self, symbol,  start_date, end_date):
        """Method returns prepared request query for Yahoo YQL API."""
        query = \
            'select * from yahoo.finance.historicaldata where symbol = "%s" and startDate = "%s" and endDate = "%s"' \
            % (symbol, start_date, end_date)

        return query

    def send(self, symbol, start_date, end_date):
        """Method sends request to Yahoo YQL API."""
        query = self.prepare_query(symbol, start_date, end_date)

        self.parameters['q'] = query
        response = requests.get(self.api, params=self.parameters).json()

        results = response['query']['results']['quote']

        return results

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta

class const(object):
    ONE_YEAR = 1

class YQL(object):
    """yql-finance is simple and fast https://developer.yahoo.com/yql/console/ python API.
    API returns stock closing prices for current period of time and current stock ticker/symbol (i.e. APPL, GOOGL).
    You can use it to fetch data in one of two ways::
        - yql = YQL('AAPL', '2011-01-01', '2014-12-31')
        or
        - yql = YQL()
          yql.select('AAPL', '2011-01-01', '2014-12-31')
    Requirements:
        - requests
        - dateutil
    """
    request = Request()

    def __repr__(self):
        return '<YQL Object: symbol %s start_date / %s end_date %s>' % (self.symbol, self.start_date, self.end_date)

    def __init__(self, symbol, start_data, end_data):
        """Method setups basic (self) variables."""
        self.start_date = self.to_date(start_data)
        self.end_date = self.to_date(end_data)
        self.symbol = symbol

        self.data = self.fetch_data()

    def __getitem__(self, index):
        return self.data[index]

    def __setitem__(self, index, value):
        self.data[index] = value

    def __delitem__(self, index):
        del self.data[index]

    @staticmethod
    def to_date(date):
        """Method converts string date to datetime date. You should use %Y-%m-%d format."""
        return datetime.datetime.strptime(date, '%Y-%m-%d').date()

    @classmethod
    def select(cls, symbol, start_date, end_date):
        """Method returns stock prices for current: ticker/symbol, start date, end date."""
        instance = cls(symbol, start_date, end_date)

        return instance.data

    def fetch_data(self):
        """Method returns results from response of Yahoo YQL API. It should returns always python list."""
        if relativedelta(self.end_date, self.start_date).years <= const.ONE_YEAR:
            data = self.request.send(self.symbol, self.start_date, self.end_date)
        else:
            data = self.fetch_chunk_data()

        return self.clean(data)

    def fetch_chunk_data(self):
        """If period of time between start end end is bigger then one year
        We have to create and fetch chunks dates (6 months chunks)."""
        data = []

        counter = (relativedelta(self.end_date, self.start_date).months / 6) + 1
        months = 0

        for month in range(counter):

            chunk_start_date = self.start_date + relativedelta(months=months)
            chunk_end_date = self.start_date + relativedelta(months=months + 6)

            months += 6

            if chunk_end_date > self.end_date:
                chunk_end_date = self.end_date

            data = data + self.request.send(self.symbol, chunk_start_date, chunk_end_date)

        return data

    def clean(self, data):
        """Method returns cleaned list of stock closing prices
        (i.e. dict(date=datetime.date(2015, 1, 2), price='23.21'))."""
        cleaned_data = list()

        if not isinstance(data, list):
            data = [data]

        for item in data:
            date = datetime.datetime.strptime(item['Date'], '%Y-%m-%d').date()
            cleaned_data.append(dict(price=item['Adj_Close'], date=date))

        return cleaned_data

In [ ]:
from rauth import OAuth1Service

try:
    read_input = raw_input
except NameError:
    read_input = input

# Get a real consumer key & secret from https://dev.twitter.com/apps/new
twitter = OAuth1Service(
    name='twitter',
    consumer_key='pWUdDZrr42vdXds2xPoFdvoKE',
    consumer_secret='5ZZNBe1wKxFEed6D2H1IPzW5HYA2ccIy9xGx0ToveF0xx5o9Gl',
    request_token_url='https://api.twitter.com/oauth/request_token',
    access_token_url='https://api.twitter.com/oauth/access_token',
    authorize_url='https://api.twitter.com/oauth/authorize',
    base_url='https://api.twitter.com/1.1/')

request_token, request_token_secret = twitter.get_request_token()

authorize_url = twitter.get_authorize_url(request_token)

print('Visit this URL in your browser: {url}'.format(url=authorize_url))
pin = read_input('Enter PIN from browser: ')

session = twitter.get_auth_session(request_token,
                                   request_token_secret,
                                   method='POST',
                                   data={'oauth_verifier': pin})

params = {'include_rts': 1,  # Include retweets
          'count': 10}       # 10 tweets

r = session.get('statuses/home_timeline.json', params=params, verify=True)

for i, tweet in enumerate(r.json(), 1):
    handle = tweet['user']['screen_name']
    text = tweet['text']
    print(u'{0}. @{1} - {2}'.format(i, handle, text))



In [78]:
from rauth import OAuth1Service

try:
    read_input = raw_input
except NameError:
    read_input = input

# Get a real consumer key & secret from https://dev.twitter.com/apps/new
twitter = OAuth1Service(
    name='yahoo',
    consumer_key='dj0yJmk9VWo0RXhCSk80MmlQJmQ9WVdrOVNXRjFOa2cwTm1zbWNHbzlNQS0tJnM9Y29uc3VtZXJzZWNyZXQmeD1hYw--',
    consumer_secret='0d0c9222ce8533a5e1350e38f9216ef633561289',
    request_token_url='https://api.login.yahoo.com/oauth/v2/get_request_token',
    access_token_url='https://api.login.yahoo.com/oauth/v2/get_token',
    authorize_url='https://api.login.yahoo.com/oauth/v2/request_auth',
    base_url='http://query.yahooapis.com/v1/yql')


request_token, request_token_secret = twitter.get_request_token()

authorize_url = twitter.get_authorize_url(request_token)

print('Visit this URL in your browser: {url}'.format(url=authorize_url))
pin = read_input('Enter PIN from browser: ')

session = twitter.get_auth_session(request_token,
                                   request_token_secret,
                                   method='POST',
                                   data={'oauth_verifier': pin})

params = {'include_rts': 1,  # Include retweets
          'count': 10}       # 10 tweets

r = session.get('statuses/home_timeline.json', params=params, verify=True)

for i, tweet in enumerate(r.json(), 1):
    handle = tweet['user']['screen_name']
    text = tweet['text']
    print(u'{0}. @{1} - {2}'.format(i, handle, text))

KeyError: "Decoder failed to handle oauth_token with data as returned by provider. A different decoder may be needed. Provider returned: b'oauth_problem=parameter_absent&oauth_parameters_absent=oauth_callback'"

In [74]:
import pdb
import json
import time
import base64
from rauth.utils import parse_utf8_qsl

class BaseOAuth():
    """
    """
    def __init__(self, oauth_version, consumer_key, consumer_secret):
        """
        consumer_key : client key
        consumer_secret : client secret
        access_token : access token
        access_token_secret : access token secret
        from_file : file containing the credentials
        base url : Base url
        """
        
        self.oauth_version = oauth_version
        self.consumer_key = consumer_key
        self.consumer_secret = consumer_secret
        
        self.oauth_version = oauth_version
        
        # Init OAuth
        if self.oauth_version == 'oauth1':
            service_params = {
                'consumer_key': self.consumer_key,
                'consumer_secret': self.consumer_secret,
                'request_token_url': "https://api.login.yahoo.com/oauth/v2/get_request_token"
            }
            
        service_params.update({
                'name' : 'yahoo',
                'access_token_url' : "https://api.login.yahoo.com/oauth/v2/get_token",    
                'authorize_url' : "https://api.login.yahoo.com/oauth/v2/request_auth",
                'base_url' : None
            })
        
        # Defining oauth service
        self.oauth = rauth.OAuth1Service(**service_params)
        
        if 
        